In [ ]:
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

# Directory to pull data from 
output_dir = 'sensor_data'

# List contents of directory
files = os.listdir(output_dir)
print(files)

# Extract frame number from file names
lidar_files = sorted([f for f in files if f.startswith('lidar_')])
camera_files = sorted([f for f in files if f.startswith('camera_')])

# Only process frames if available
frame_numbers = sorted(set(int(f.split('_')[1].split('.')[0]) for f in lidar_files) &
                       set(int(f.split('_')[1].split('.')[0]) for f in camera_files))

# Load saved data
def load_data(frame_number):
    lidar_file = os.path.join(output_dir, f'lidar_{frame_number}.npy')
    camera_file = os.path.join(output_dir, f'camera_{frame_number}.npy')
    
    lidar_data = np.load(lidar_file)
    camera_data = np.load(camera_file)
    return lidar_data, camera_data

# Init visual odometry
orb = cv2.ORB_create()
bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
trajectory = []

# Process each frame based on availability
prev_img = None
prev_kp = None
prev_des = None
position = np.array([0, 0], dtype=np.float32)  # Initial pos

for frame_number in frame_numbers:
    try:
        lidar_data, camera_data = load_data(frame_number)
    except FileNotFoundError:
        continue  # Skips not found

    # Convert camera data to grayscale for feature detecting
    current_img = cv2.cvtColor(camera_data, cv2.COLOR_BGR2GRAY)
    current_kp, current_des = orb.detectAndCompute(current_img, None)

    if prev_img is not None:
        # Match previous and current image features
        matches = bf.match(prev_des, current_des)
        matches = sorted(matches, key=lambda x: x.distance)
        
        # Extract location of matched keypoints
        prev_pts = np.float32([prev_kp[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
        curr_pts = np.float32([current_kp[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
        
        # Estimate motion of camera
        E, mask = cv2.findEssentialMat(curr_pts, prev_pts, focal=1.0, pp=(0, 0), method=cv2.RANSAC, prob=0.999, threshold=1.0)
        _, R, t, mask = cv2.recoverPose(E, curr_pts, prev_pts)
        
        # Update pos
        scale = 1.0  # fixed scale
        position += scale * t[:2].flatten()

        # Append new pos to trajectory
        trajectory.append(position.copy())

    # Visualize camera image
    plt.imshow(cv2.cvtColor(camera_data, cv2.COLOR_BGR2RGB))
    plt.title(f'Camera Image - Frame {frame_number}')
    plt.show()
    plt.pause(0.1)

    # Plot LIDAR data
    plt.scatter(lidar_data[:, 0], lidar_data[:, 1], c='r', marker='.')
    plt.title(f'LIDAR Data - Frame {frame_number}')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.show()
    plt.pause(0.1)  

    # Update previous image and keypoints
    prev_img = current_img
    prev_kp = current_kp
    prev_des = current_des

# Plot final trajectory
trajectory = np.array(trajectory)
plt.plot(trajectory[:, 0], trajectory[:, 1], '-o')
plt.title('Trajectory')
plt.xlabel('X')
plt.ylabel('Y')
plt.show()
